# Logging Basics

`rubicon` can help us create an optimal machine learning model by logging the details of each
experiment we run along the way. To illustrate, we'll train a classification model using `scikit-learn`'s
[wine dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html).

Let's load the dataset and take a quick look at it's contents.

In [1]:
from sklearn.datasets import load_wine


wine = load_wine(as_frame=True)
print(wine["DESCR"][1952:2412])

The data is the results of a chemical analysis of wines grown in the same
region in Italy by three different cultivators. There are thirteen different
measurements taken for different constituents found in the three types of
wine.

Original Owners: 

Forina, M. et al, PARVUS - 
An Extendible Package for Data Exploration, Classification and Correlation. 
Institute of Pharmaceutical and Food Analysis and Technologies,
Via Brigata Salerno, 16147 Genoa, Italy.


To see the full description of the dataset use ``print(wine["DESCR"])``.

Since we specified ``as_frame=True`` when loading the dataset, we've got the data as
a ``pandas`` dataframe.

In [2]:
wine_data = wine.data
wine_data.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


Each sample holds measurements of different qualities of the wines observed. We'll train on 75%
of the data and use the other 25% to make predictions using our model.

First we'll split the data to define our training and testing subsets.

In [3]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    wine["data"],
    wine["target"],
    test_size=0.25,
)

Let's try to train a model using ``scikit-learn``'s
[random forest classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)!


**For this basic example, our goal will be to evaluate how the n_estimators parameter affects our results.**

In [4]:
from sklearn.ensemble import RandomForestClassifier


def fit_and_score_classifier(X_train, X_test, y_train, y_test, n_estimators):
    rfc = RandomForestClassifier(n_estimators=n_estimators)
    rfc.fit(X_train, y_train)
    
    accuracy = rfc.score(X_test, y_test)
    
    return accuracy

With ``rubicon``, we can create a **project** to store as many **experiments** as we'd
like to run. Each of these **experiments** can hold important metadata about your
model run - like our varied values of ``n_estimators``.

In [5]:
import os

from rubicon_ml import Rubicon


root_dir = os.environ.get("RUBICON_ROOT", "rubicon-root")
root_path = f"{os.path.dirname(os.getcwd())}/{root_dir}"

rubicon = Rubicon(persistence="filesystem", root_dir=root_path)
project = rubicon.get_or_create_project(
    "Logging Basics",
    description=(
        "to determine what values of n_estimators create ",
        "the best random forest classifier using ",
        "scikit-learn's wine dataset"
    ),
)
project

Let's get started with our first **experiment**! We can provide some basic info here like
the name of the model we're using, some metadata around where our training data came from,
and tags to easily filter our experiments later.

In [6]:
experiment = project.log_experiment(
    model_name=RandomForestClassifier.__name__,
    tags=["wine"],
    training_metadata=[("sklearn.datasets", "load_iris")],
)
experiment

We're mainly interested in how ``n_estimators`` affects our accuracy, so let's
choose a starting value to test and log it as a **parameter**.

In [7]:
n_estimators = 1

parameter = experiment.log_parameter("n_estimators", n_estimators)
parameter

We can log the list of **features** in the training dataset to ``rubicon`` as well.

In [8]:
for feature_name in wine.feature_names:
    experiment.log_feature(feature_name)
    
experiment.features()

Now we're ready to train and score our classifier - let's see how it does with
``n_estimators`` set to 1!

In [9]:
accuracy = fit_and_score_classifier(X_train, X_test, y_train, y_test, n_estimators)
accuracy

0.8888888888888888

Finally, we can log our accuracy to ``rubicon`` as a **metric**.

In [10]:
metric = experiment.log_metric("accuracy", accuracy)
metric

**Experiments** can be tagged after the fact as well. We can define some kind of
"success" criteria and tag accordingly.

In [11]:
if accuracy > 0.90:
    experiment.add_tags(["success"])

We can re-run the whole process above a few times for different values of ``n_estimators`` to
really get an idea of how the **parameter** affects our accuracy **metric**.

In [12]:
for n_estimators in [5, 10, 15, 20]:
    experiment = project.log_experiment(
        model_name=RandomForestClassifier.__name__,
        tags=["wine"],
        training_metadata=[("sklearn.datasets", "load_iris")],
    )
    
    experiment.log_parameter("n_estimators", n_estimators)
    
    for feature_name in wine.feature_names:
        experiment.log_feature(feature_name)
        
    accuracy = fit_and_score_classifier(X_train, X_test, y_train, y_test, n_estimators)
    
    experiment.log_metric("accuracy", accuracy)
    
    if accuracy > 0.90:
        experiment.add_tags(["success"])

Now we can pull all our metadata back out of ``rubicon`` to inspect the results!

In [13]:
for experiment in project.experiments():
    print((
        f"{experiment.parameters()[0].name}: {experiment.parameters()[0].value:02}\t"
        f"{experiment.metrics()[0].name}: {experiment.metrics()[0].value}"
    ))

n_estimators: 10	accuracy: 0.9555555555555556
n_estimators: 20	accuracy: 0.9777777777777777
n_estimators: 15	accuracy: 0.9555555555555556
n_estimators: 01	accuracy: 0.8888888888888888
n_estimators: 05	accuracy: 0.9333333333333333


A quick look at our logged data shows that all the values of ``n_estimators`` above 1
meet our success criteria! While this is a simple example, it shows how we can use `rubicon`
to track our model's performance over time as we try different **parameters** to optimize
our **metrics**.

`rubicon` supports even more logging capabilities, like logging 
[**artifacts**](https://capitalone.github.io/rubicon/glossary.html#artifact-rubicon-artifact)
and [**dataframes**](https://capitalone.github.io/rubicon/glossary.html#dataframe-rubicon-dataframe),
to ensure complete reproducibility. We can also use ``rubicon``'s
[dashboard](https://capitalone.github.io/rubicon/dashboard.html)
for better visual representation of your logged data.